<a href="https://colab.research.google.com/github/iliaxant/DIP_HW_1_HeatCamera/blob/main/DIP_HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>